Objetivo: Modelo de Machine Learning capaz de predecir el riesgo de enfermedad cardiovascular en los siguientes 10 años a partir de variables clínicas.

Población: personas entre 30 y 75 años en 2012, con tarjeta sanitaria, no fallecidas actualmente y sin diagnósticos de enfermedades cardiovasculares anteriores a 2012.

Variable dependiente: ¿ha sufrido alguna enfermedad cardiovascular en los siguientes 10 años?

Variables independientes: diagnóstico de diabetes, diagnóstico o hábito de tabaquismo, diagnóstico de hipercolesterolemia, diagnóstico de hipertensión, edad, sexo, constante HDL, constante LDL, constante colesterol total, constante de albúmina, constante de cociente de albúmina/creatinina, constante de TAD, constante de TAS, diagnóstico de hipertrofia ventricular izquierda, antecedente familiar de ECV.

In [1]:
from pyspark.sql.functions import floor, datediff, lit, rand, monotonically_increasing_id, exists, date_add, when, col, to_date, substring, expr, countDistinct, current_date, max
from pyspark.sql.types import IntegerType

In [2]:
from utils_spark import UtilsSpark
utilsSpark = UtilsSpark(True, app_name='modelo_framingham')
from utils_diagnosticos import UtilsDiagnosticos
from utils_tarjeta import UtilsTarjeta

In [3]:
driver = 'oracle.jdbc.driver.OracleDriver'
url = 'jdbc:oracle:thin:@10.136.1.89:1526/ORAIX1'
user = 'siap_exp'
password = 'sexp2013'

In [4]:
utilsDiagnosticos = UtilsDiagnosticos(True, db_connection={'driver': driver, 'url': url, 'user': user, 'password': password})
utilsTarjeta = UtilsTarjeta(True, db_connection={'driver': driver, 'url': url, 'user': user, 'password': password})

La fecha de estudio será 01/12/2012, 10 años antes de la fecha actual

In [5]:
fecha_estudio = '2012-12-01'

Obtenemos personas con TS actualmente

In [6]:
tarjeta = utilsTarjeta.pacientes_tarjeta()

No quedamos con las personas que tienen entre 40 y 85 años actualmente, es decir, las que tenían entre 30 y 75 años hace 10 años

In [7]:
tarjeta_edad = tarjeta.where(tarjeta['edad'].between(40, 85))
#print(f'Población con TS de entre 40 y 85 años: {str(tarjeta_edad.count())}')

Estos son los diagnósticos de enfermedades cardiovasculares

In [8]:
diag_ecv_csv_path = 'dbfs:/FileStore/otros/Diagnósticos_Enfermedad_Cardiovascular.csv'
#diag_ecv_csv_path = './Diagnósticos Enfermedad Cardiovascular.csv'

In [9]:
diag_ecv = utilsSpark.spark.read.csv(diag_ecv_csv_path, header=True, sep=';')
diag_ecv = diag_ecv.withColumn('id', diag_ecv['id'].cast(IntegerType()))
diag_ecv = diag_ecv.select('id', 'cie9')

Obtenemos todos los pacientes con un diagnóstico activo o no de ECV anterior a 01/12/2012

In [10]:
#pacientes_ecv_cie9 = utilsDiagnosticos.pacientes_con_diagnostico_cie9(diagnosticos=diag_ecv, columna_diagnostico='cie9', fecha_creado=lit(fecha_estudio))
pacientes_ecv_proceso = utilsDiagnosticos.pacientes_con_diagnostico_proceso(diagnosticos=diag_ecv, columna_diagnostico='id', fecha_creado=lit(fecha_estudio))
#pacientes_ecv = pacientes_ecv_cie9.union(pacientes_ecv_proceso)
pacientes_ecv = pacientes_ecv_proceso
pacientes_ecv = pacientes_ecv.orderBy('fec_activacion', ascending=True)
pacientes_ecv = pacientes_ecv.dropDuplicates(['num_expediente', 'proceso', 'ccie9'])

Descartamos la población que se encuentra en este grupo de pacientes con un diagnóstico de ECV

In [11]:
poblacion_objetivo = tarjeta_edad.join(pacientes_ecv, tarjeta_edad['num_expediente'] == pacientes_ecv['num_expediente'], how='left_anti')

Esta es la población objetivo de estudio. Ahora, para esta población, debemos calcular las variables mencionadas al comienzo

Estas variables tienen que haber sido tomadas en un tiempo cercano a la fecha de estudio porque, sino, aumenta el margen de error con la situación real del paciente justo en la fecha de estudio

In [12]:
fecha_min_variables = '2011-12-01'

Vamos primero a obtener las constantes (HDL, LDL, TAS, TAD, etc.)

In [13]:
constantes_estudio = ['HDL-Colesterol', 'LDL-Colesterol', 'COLESTEROL', 'COCIENTE ALBUMINA/CREATININA', 'TAD', 'TAS']

In [14]:
explora = utilsSpark.obtener_tabla(driver, url, user, password, 'explora')
explora = explora.where(explora['codigo'].isin(constantes_estudio))

Nos quedamos con las constantes de los pacientes objetivo

In [15]:
explora_join = poblacion_objetivo.select('num_expediente')
explora = explora.join(explora_join, explora['num_expediente'] == explora_join['num_expediente']).drop(explora_join['num_expediente'])

De estas constantes, nos quedamos con las tomadas entre la fecha de estudio y un año antes

In [16]:
explora = explora.where((explora['fecha'] >= lit(fecha_min_variables)) & (explora['fecha'] <= lit(fecha_estudio)))

In [17]:
explora = explora.select('num_expediente', 'fecha', 'codigo', 'valor', 'hora')

De estas, para cada constante y para cada persona, nos quedamos con la última, la más cercana a la fecha de estudio

In [18]:
explora = explora.orderBy(['num_expediente', 'codigo', 'fecha', 'hora'], ascending=[True, True, False, False])
explora_ultimo_valor = explora.dropDuplicates(['num_expediente', 'codigo', 'fecha'])
explora_ultimo_valor = explora_ultimo_valor.dropDuplicates(['num_expediente', 'codigo'])
explora_ultimo_valor = explora_ultimo_valor.cache()

Añadimos estos valores a la población objetivo sin hacer un inner join, colocando NULL donde no hayan valores disponibles

In [19]:
hdl = explora_ultimo_valor.where(explora_ultimo_valor['codigo'] == 'HDL-Colesterol').select('num_expediente', 'valor')
ldl = explora_ultimo_valor.where(explora_ultimo_valor['codigo'] == 'LDL-Colesterol').select('num_expediente', 'valor')
colesterol = explora_ultimo_valor.where(explora_ultimo_valor['codigo'] == 'COLESTEROL').select('num_expediente', 'valor')
coc_alb_crea = explora_ultimo_valor.where(explora_ultimo_valor['codigo'] == 'COCIENTE ALBUMINA/CREATININA').select('num_expediente', 'valor')
tad = explora_ultimo_valor.where(explora_ultimo_valor['codigo'] == 'TAD').select('num_expediente', 'valor')
tas = explora_ultimo_valor.where(explora_ultimo_valor['codigo'] == 'TAS').select('num_expediente', 'valor')

In [20]:
hdl = hdl.withColumnRenamed('valor', 'hdl').withColumnRenamed('num_expediente', 'num_expediente_hdl')
ldl = ldl.withColumnRenamed('valor', 'ldl').withColumnRenamed('num_expediente', 'num_expediente_ldl')
colesterol = colesterol.withColumnRenamed('valor', 'colesterol').withColumnRenamed('num_expediente', 'num_expediente_colesterol')
coc_alb_crea = coc_alb_crea.withColumnRenamed('valor', 'coc_alb_crea').withColumnRenamed('num_expediente', 'num_expediente_coc_alb_crea')
tad = tad.withColumnRenamed('valor', 'tad').withColumnRenamed('num_expediente', 'num_expediente_tad')
tas = tas.withColumnRenamed('valor', 'tas').withColumnRenamed('num_expediente', 'num_expediente_tas')

In [21]:
poblacion_objetivo = poblacion_objetivo.join(hdl, poblacion_objetivo['num_expediente'] == hdl['num_expediente_hdl'], how='left').drop(hdl['num_expediente_hdl'])

In [22]:
poblacion_objetivo = poblacion_objetivo.join(ldl, poblacion_objetivo['num_expediente'] == ldl['num_expediente_ldl'], how='left').drop(ldl['num_expediente_ldl'])

In [23]:
poblacion_objetivo = poblacion_objetivo.join(colesterol, poblacion_objetivo['num_expediente'] == colesterol['num_expediente_colesterol'], how='left').drop(colesterol['num_expediente_colesterol'])

In [24]:
poblacion_objetivo = poblacion_objetivo.join(coc_alb_crea, poblacion_objetivo['num_expediente'] == coc_alb_crea['num_expediente_coc_alb_crea'], how='left').drop(coc_alb_crea['num_expediente_coc_alb_crea'])

In [25]:
poblacion_objetivo = poblacion_objetivo.join(tad, poblacion_objetivo['num_expediente'] == tad['num_expediente_tad'], how='left').drop(tad['num_expediente_tad'])

In [26]:
poblacion_objetivo = poblacion_objetivo.join(tas, poblacion_objetivo['num_expediente'] == tas['num_expediente_tas'], how='left').drop(tas['num_expediente_tas'])

Ahora realicemos un proceso similar para calcular el resto de variables

Calculemos las personas que tenían Diabetes antes de la fecha de estudio

In [27]:
circun_clinicas_dx = utilsSpark.obtener_tabla(driver, url, user, password, 'siap_pro.vw_ccc_circun_clinicas_dx')

In [28]:
circun_clinicas_dx_diabetes = circun_clinicas_dx.where(circun_clinicas_dx['des_cc'].isin(['DIABETES NO INCLUIDA (1,2 Y GESTACIONAL)', 'DIABETES TIPO 1', 'DIABETES TIPO 2']))
circun_clinicas_dx_diabetes = circun_clinicas_dx_diabetes.select(['codigo'])

In [29]:
hi_enfermedades = utilsSpark.obtener_tabla(driver, url, user, password, 'hi_enfermedades')

In [30]:
hi_enfermedades_diabetes = hi_enfermedades.join(circun_clinicas_dx_diabetes, hi_enfermedades['cie9'] == circun_clinicas_dx_diabetes['codigo']).select(['id', 'cie9']).distinct()

In [31]:
diabetes_cie9 = utilsDiagnosticos.pacientes_con_diagnostico_cie9(hi_enfermedades_diabetes, columna_diagnostico='cie9', fecha_creado=lit(fecha_estudio))
diabetes_proceso = utilsDiagnosticos.pacientes_con_diagnostico_proceso(hi_enfermedades_diabetes, columna_diagnostico='id', fecha_creado=lit(fecha_estudio))
diabetes = diabetes_cie9.union(diabetes_proceso)
diabetes = diabetes.orderBy('fec_activacion', ascending=True)
diabetes = diabetes.dropDuplicates(['num_expediente'])

In [32]:
diabetes = diabetes.withColumn('diabetes', lit(1)).select('num_expediente', 'diabetes')
poblacion_objetivo = poblacion_objetivo.join(diabetes, poblacion_objetivo['num_expediente'] == diabetes['num_expediente'], how='left').drop(diabetes['num_expediente'])
poblacion_objetivo = poblacion_objetivo.withColumn('diabetes', when(poblacion_objetivo['diabetes'] == 1, lit(1)).otherwise(lit(0)))

Calculemos las personas que tenían Hipertensión antes de la fecha de estudio

In [33]:
circun_clinicas_dx_hta = circun_clinicas_dx.where(circun_clinicas_dx['des_cc'].isin(['HIPERTENSIÓN ARTERIAL']))
circun_clinicas_dx_hta = circun_clinicas_dx_hta.select(['codigo'])

In [34]:
hi_enfermedades_hta = hi_enfermedades.join(circun_clinicas_dx_hta, hi_enfermedades['cie9'] == circun_clinicas_dx_hta['codigo']).select(['id', 'cie9']).distinct()

In [35]:
hipertension_cie9 = utilsDiagnosticos.pacientes_con_diagnostico_cie9(hi_enfermedades_hta, columna_diagnostico='cie9', fecha_creado=lit(fecha_estudio))
hipertension_proceso = utilsDiagnosticos.pacientes_con_diagnostico_proceso(hi_enfermedades_hta, columna_diagnostico='id', fecha_creado=lit(fecha_estudio))
hipertension = hipertension_cie9.union(hipertension_proceso)
hipertension = hipertension.orderBy('fec_activacion', ascending=True)
hipertension = hipertension.dropDuplicates(['num_expediente'])

In [36]:
hipertension = hipertension.withColumn('hipertension', lit(1)).select('num_expediente', 'hipertension')
poblacion_objetivo = poblacion_objetivo.join(hipertension, poblacion_objetivo['num_expediente'] == hipertension['num_expediente'], how='left').drop(hipertension['num_expediente'])
poblacion_objetivo = poblacion_objetivo.withColumn('hipertension', when(poblacion_objetivo['hipertension'] == 1, lit(1)).otherwise(lit(0)))

Calculemos las personas que tenían Hipercolesterolemia antes de la fecha de estudio

In [37]:
hi_enfermedades_hipercol = hi_enfermedades.where(hi_enfermedades['cie9'].isin(['272.0', '272.2'])).select(['id', 'cie9']).distinct()

In [38]:
hipercolesterolemia_cie9 = utilsDiagnosticos.pacientes_con_diagnostico_cie9(hi_enfermedades_hipercol, columna_diagnostico='cie9', fecha_creado=lit(fecha_estudio))
hipercolesterolemia_proceso = utilsDiagnosticos.pacientes_con_diagnostico_proceso(hi_enfermedades_hipercol, columna_diagnostico='id', fecha_creado=lit(fecha_estudio))
hipercolesterolemia = hipercolesterolemia_cie9.union(hipercolesterolemia_proceso)
hipercolesterolemia = hipercolesterolemia.orderBy('fec_activacion', ascending=True)
hipercolesterolemia = hipercolesterolemia.dropDuplicates(['num_expediente'])

In [39]:
hipercolesterolemia = hipercolesterolemia.withColumn('hipercolesterolemia', lit(1)).select('num_expediente', 'hipercolesterolemia')
poblacion_objetivo = poblacion_objetivo.join(hipercolesterolemia, poblacion_objetivo['num_expediente'] == hipercolesterolemia['num_expediente'], how='left').drop(hipercolesterolemia['num_expediente'])
poblacion_objetivo = poblacion_objetivo.withColumn('hipercolesterolemia', when(poblacion_objetivo['hipercolesterolemia'] == 1, lit(1)).otherwise(lit(0)))

Calculemos las personas que tenían Tabaquismo antes de la fecha de estudio. Este es un poco diferente porque también hay que mirar en hábitos

In [40]:
circun_clinicas_dx_tabaq = circun_clinicas_dx.where(circun_clinicas_dx['des_cc'].isin(['TABAQUISMO']))
circun_clinicas_dx_tabaq = circun_clinicas_dx_tabaq.select(['codigo'])

In [41]:
hi_enfermedades_tabaq = hi_enfermedades.join(circun_clinicas_dx_tabaq, hi_enfermedades['cie9'] == circun_clinicas_dx_tabaq['codigo']).select(['id', 'cie9']).distinct()

In [42]:
tabaquismo_cie9 = utilsDiagnosticos.pacientes_con_diagnostico_cie9(hi_enfermedades_tabaq, columna_diagnostico='cie9', fecha_creado=lit(fecha_estudio))
tabaquismo_proceso = utilsDiagnosticos.pacientes_con_diagnostico_proceso(hi_enfermedades_tabaq, columna_diagnostico='id', fecha_creado=lit(fecha_estudio))
tabaquismo_diag = tabaquismo_cie9.union(tabaquismo_proceso)
tabaquismo_diag = tabaquismo_diag.orderBy('fec_activacion', ascending=True)
tabaquismo_diag = tabaquismo_diag.dropDuplicates(['num_expediente']).select('num_expediente')

In [43]:
habitusu = utilsSpark.obtener_tabla(driver, url, user, password, 'habitusu')
tabaquismo_hab = habitusu.where(habitusu['codigo'] == 'TABACO')
tabaquismo_hab = tabaquismo_hab.where(tabaquismo_hab['valor'].isNotNull())
tabaquismo_hab = tabaquismo_hab.where(tabaquismo_hab['valor'] > 0)
tabaquismo_hab = tabaquismo_hab.withColumn('fecha_no_null', when(tabaquismo_hab['fecha'].isNull(), tabaquismo_hab['fecact']).otherwise(tabaquismo_hab['fecha']))
tabaquismo_hab = tabaquismo_hab.where(tabaquismo_hab['fecha_no_null'] <= lit(fecha_estudio))
tabaquismo_hab = tabaquismo_hab.select('num_expediente')

In [44]:
tabaquismo = tabaquismo_diag.union(tabaquismo_hab)
tabaquismo = tabaquismo.withColumn('tabaquismo', lit(1))
tabaquismo = tabaquismo.select('num_expediente', 'tabaquismo').distinct()
poblacion_objetivo = poblacion_objetivo.join(tabaquismo, poblacion_objetivo['num_expediente'] == tabaquismo['num_expediente'], how='left').drop(tabaquismo['num_expediente'])
poblacion_objetivo = poblacion_objetivo.withColumn('tabaquismo', when(poblacion_objetivo['tabaquismo'] == 1, lit(1)).otherwise(lit(0)))

Calculemos las personas que tenían Hipertrofia Ventricular izquierda antes de la fecha de estudio.

In [45]:
diag_hvi = [16655, 127645, 183604, 127646, 127643, 169129, 16703, 15551, 187328, 16656, 16704, 187967, 146243, 146244, 195673]

In [46]:
hipertrofia_vent_izq = utilsDiagnosticos.pacientes_con_diagnostico_proceso(diag_hvi, fecha_creado=lit(fecha_estudio))
hipertrofia_vent_izq = hipertrofia_vent_izq.orderBy('fec_activacion', ascending=True)
hipertrofia_vent_izq = hipertrofia_vent_izq.dropDuplicates(['num_expediente'])

In [47]:
hipertrofia_vent_izq = hipertrofia_vent_izq.withColumn('hipertrofia_vent_izq', lit(1)).select('num_expediente', 'hipertrofia_vent_izq')
poblacion_objetivo = poblacion_objetivo.join(hipertrofia_vent_izq, poblacion_objetivo['num_expediente'] == hipertrofia_vent_izq['num_expediente'], how='left').drop(hipertrofia_vent_izq['num_expediente'])
poblacion_objetivo = poblacion_objetivo.withColumn('hipertrofia_vent_izq', when(poblacion_objetivo['hipertrofia_vent_izq'] == 1, lit(1)).otherwise(lit(0)))

Ahora los antecedentes familiares de RCV. Aquí necesitamos antefami

In [48]:
antefami = utilsSpark.obtener_tabla(driver, url, user, password, 'antefami')
antefami_ecv_proceso = antefami.join(diag_ecv, antefami['proceso'] == diag_ecv['id'])
antefami_ecv_cie9 = antefami.join(diag_ecv, antefami['causa'] == diag_ecv['cie9'])
antefami_ecv = antefami_ecv_cie9.union(antefami_ecv_proceso)
antefami_ecv = antefami_ecv.where(antefami_ecv['fecha'] <= lit(fecha_estudio))
antefami_ecv = antefami_ecv.select('contador_usu').distinct()
antefami_ecv = antefami_ecv.withColumnRenamed('contador_usu', 'num_expediente')
antefami_ecv = antefami_ecv.withColumn('antefami_ecv', lit(1))

In [49]:
poblacion_objetivo = poblacion_objetivo.join(antefami_ecv, poblacion_objetivo['num_expediente'] == antefami_ecv['num_expediente'], how='left').drop(antefami_ecv['num_expediente'])
poblacion_objetivo = poblacion_objetivo.withColumn('antefami_ecv', when(poblacion_objetivo['antefami_ecv'] == 1, lit(1)).otherwise(lit(0)))

Por último, queda la variable dependiente, si han padecido o no una ECV en los diez años siguientes a la fecha de estudio

In [50]:
#despues_10_ecv_cie9 = utilsDiagnosticos.pacientes_con_diagnostico_cie9(diagnosticos=diag_ecv, columna_diagnostico='cie9')
despues_10_ecv_proceso = utilsDiagnosticos.pacientes_con_diagnostico_proceso(diagnosticos=diag_ecv, columna_diagnostico='id')

In [51]:
#despues_10_ecv_cie9 = despues_10_ecv_cie9.where(despues_10_ecv_cie9['fec_activacion'] > lit(fecha_estudio))
despues_10_ecv_proceso = despues_10_ecv_proceso.where(despues_10_ecv_proceso['fec_activacion'] > lit(fecha_estudio))

In [52]:
#despues_10_ecv = despues_10_ecv_cie9.union(despues_10_ecv_proceso)
despues_10_ecv = despues_10_ecv_proceso
despues_10_ecv = despues_10_ecv.orderBy('fec_activacion', ascending=True)
despues_10_ecv = despues_10_ecv.dropDuplicates(['num_expediente'])

In [53]:
despues_10_ecv = despues_10_ecv.withColumn('ecv_10_anos_despues', lit(1)).select('num_expediente', 'ecv_10_anos_despues')
poblacion_objetivo = poblacion_objetivo.join(despues_10_ecv, poblacion_objetivo['num_expediente'] == despues_10_ecv['num_expediente'], how='left').drop(despues_10_ecv['num_expediente'])
poblacion_objetivo = poblacion_objetivo.withColumn('ecv_10_anos_despues', when(poblacion_objetivo['ecv_10_anos_despues'] == 1, lit(1)).otherwise(lit(0)))

Teniendo ya todos los datos necesarios, comenzaremos a preparar la exportación

Primero nos aseguraremos de que no hay más de un registro para cada paciente

In [54]:
#poblacion_objetivo = poblacion_objetivo.persist()

In [55]:
#poblacion_objetivo.count()

In [56]:
#poblacion_objetivo.select('num_expediente').distinct().count()

Ahora codificaremos la identificación de cada paciente y un ajuste a la edad

In [57]:
poblacion_objetivo = poblacion_objetivo.withColumn('id', monotonically_increasing_id())
poblacion_objetivo = poblacion_objetivo.withColumn('edad_hace_10_anos', poblacion_objetivo['edad'] - 10)

In [58]:
poblacion_objetivo = poblacion_objetivo.select('id', 'num_expediente', 'ecv_10_anos_despues', 'cod_sexo', 'edad_hace_10_anos', 'diabetes', 'hipertension', 'hipercolesterolemia', 'tabaquismo',
                                               'hipertrofia_vent_izq', 'antefami_ecv', 'hdl', 'ldl', 'colesterol', 'coc_alb_crea', 'tad', 'tas')
poblacion_objetivo = poblacion_objetivo.orderBy('num_expediente')

Exportamos

In [59]:
utilsSpark.exportar_csv(poblacion_objetivo, 'Poblacion Objetivo')

Comienzo: 2022-12-21 14:11:45.246459
Fin: 2022-12-21 14:52:28.214488
Tiempo empleado:0:40:42.968029


In [ ]:
utilsSpark.spark.stop()